<a href="https://colab.research.google.com/github/arkothiwala/deep-learning/blob/master/AV_Intel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
import zipfile
zip_ref = zipfile.ZipFile('train-scene classification.zip', 'r')
zip_ref.extractall('data')
zip_ref.close()

In [0]:
from fastai.vision import *
from fastai.metrics import error_rate
import fastai

In [0]:
#batch size
bs = 64

In [0]:
path_img = 'data/train'
fnames = get_image_files(path_img)

In [20]:
fnames[:5]

[PosixPath('data/train/18693.jpg'),
 PosixPath('data/train/11832.jpg'),
 PosixPath('data/train/12212.jpg'),
 PosixPath('data/train/12191.jpg'),
 PosixPath('data/train/12072.jpg')]

In [34]:
help(ImageDataBunch.from_csv)

Help on method from_csv in module fastai.vision.data:

from_csv(path:Union[pathlib.Path, str], folder:Union[pathlib.Path, str]='.', label_delim:str=None, csv_labels:Union[pathlib.Path, str]='labels.csv', valid_pct:float=0.2, fn_col:int=0, label_col:int=1, suffix:str='', header:Union[int, str, NoneType]='infer', **kwargs:Any) -> 'ImageDataBunch' method of builtins.type instance
    Create from a csv file in `path/csv_labels`.



In [0]:
data = ImageDataBunch.from_csv(path_img, ds_tfms=get_transforms(), csv_labels='../train.csv', size=150, bs=bs).normalize(imagenet_stats)

In [38]:
data.batch_size

64